In [24]:
import pandas as pd
from pathlib import Path

In [25]:
SOURCE_DIR = Path('../files/output/csv_output/treat_runs/')
SOURCE_CSV = Path(SOURCE_DIR/'treat3run_filt.csv')
VALIDATED_ID_CSV = Path(SOURCE_DIR/'fixed_ids.csv')
INPUT_CSV = Path(SOURCE_DIR/'routes_to_fix.csv')
OUTPUT_CSV = Path(SOURCE_DIR/'all_direct_valid_routes.csv')
OUTPUT_CSV_2 = Path('../output/csv_output/all_direct_valid_routes.csv')

In [26]:
if not OUTPUT_CSV.is_file():
    df_source = pd.read_csv(SOURCE_CSV, index_col='path_id')
    df_source.to_csv(OUTPUT_CSV)
df_output = pd.read_csv(OUTPUT_CSV, index_col='path_id')
df_output_2 = pd.read_csv(OUTPUT_CSV_2, index_col='path_id')

In [27]:
df_input = pd.read_csv(INPUT_CSV, usecols=['from_id', 'to_id', 'transport_id', 'price_min_EUR'])
df_input

,from_id,to_id,transport_id,price_min_EUR
0,336,52,3,6
1,336,52,2,4
2,336,171,2,15
3,52,171,2,18


In [28]:
if not VALIDATED_ID_CSV.is_file():
    with open(VALIDATED_ID_CSV, 'w') as val_file:
        val_file.write('path_id')
df_validated = pd.read_csv(VALIDATED_ID_CSV, index_col=None)

In [29]:
valid_trips = tuple(zip(df_input['from_id'], df_input['to_id'], df_input['transport_id'], df_input['price_min_EUR']))
valid_trips

((336, 52, 3, 6), (336, 52, 2, 4), (336, 171, 2, 15), (52, 171, 2, 18))

In [30]:

for valid_trip in valid_trips:
    # forward
    forward_query = 'from_id == @valid_trip[0] and to_id == @valid_trip[1] and transport_id == @valid_trip[2]'
    match_index = df_output.query(forward_query).index
    if match_index not in list(df_validated['path_id']):
        df_output.at[match_index, 'price_min_EUR'] = valid_trip[3]
        df_validated.at[len(df_validated.index)] = match_index
    
    # backward
    backward_query = 'from_id == @valid_trip[1] and to_id == @valid_trip[0] and transport_id == @valid_trip[2]'
    match_index = df_output.query(backward_query).index
    if match_index not in list(df_validated['path_id']):
        df_output.at[match_index, 'price_min_EUR'] = valid_trip[3]
        df_validated.at[len(df_validated.index)] = match_index
  

In [31]:
df_output.to_csv(OUTPUT_CSV)
df_validated.to_csv(VALIDATED_ID_CSV, index=None)